# Spark SQLContext

## Documentation: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html

Initilizing SQLContext

In [1]:
!rm -rf metastore_db/
import pyspark
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

## Creating user-defined functions

To use RDD in SQLContext, RDD lines have to be converted into a Row format

In [2]:
import re
from pyspark.sql import Row
# Read from CSV
def load_csv(line):
    return re.split("\"?,\"?", line)
        
def readInt(x):
    try:
        return int(x)
    except:
        return x

def parseElement(e):
    return Row(year=readInt(e[0]),
    month=readInt(e[1]),
    day=readInt(e[2]),
    dep_time=readInt(e[3]),
    dep_delay=readInt(e[4]),
    arr_time=readInt(e[5]),
    arr_delay=readInt(e[6]),
    cancelled=e[7],
    carrier=e[8],
    tailnum=e[9],
    flight=readInt(e[10]),
    origin=e[11],
    dest=e[12],
    air_time=readInt(e[13]),
    distance=readInt(e[14]),
    hour=readInt(e[15]),
    min=readInt(e[16])
    )

In [3]:
!wget https://dsr-data.s3.amazonaws.com/flights/flights14.csv

--2017-01-17 22:07:59--  https://dsr-data.s3.amazonaws.com/flights/flights14.csv
Resolving dsr-data.s3.amazonaws.com (dsr-data.s3.amazonaws.com)... 54.231.194.32
Connecting to dsr-data.s3.amazonaws.com (dsr-data.s3.amazonaws.com)|54.231.194.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16150465 (15M) [text/csv]
Saving to: ‘flights14.csv’

100%[======================================>] 16.150.465  5,39MB/s   in 2,9s   

2017-01-17 22:08:02 (5,39 MB/s) - ‘flights14.csv’ saved [16150465/16150465]



In [4]:
flights = sc.textFile("flights14.csv").map(load_csv).filter(lambda e: not e[0] == "year").map(parseElement).cache()

Now the schema and the RDD have to be registered with the sqlContext:

In [5]:
flightsDF = sqlContext.createDataFrame(flights)

In [6]:
flightsDF.registerTempTable("flight")

In [7]:
sqlContext.sql("select * from flight where dest = 'LAX'").take(5)

[Row(air_time=359, arr_delay=13, arr_time=1238, cancelled=u'0', carrier=u'AA', day=1, dep_delay=14, dep_time=914, dest=u'LAX', distance=2475, flight=1, hour=9, min=14, month=1, origin=u'JFK', tailnum=u'N338AA', year=2014),
 Row(air_time=363, arr_delay=13, arr_time=1523, cancelled=u'0', carrier=u'AA', day=1, dep_delay=-3, dep_time=1157, dest=u'LAX', distance=2475, flight=3, hour=11, min=57, month=1, origin=u'JFK', tailnum=u'N335AA', year=2014),
 Row(air_time=351, arr_delay=9, arr_time=2224, cancelled=u'0', carrier=u'AA', day=1, dep_delay=2, dep_time=1902, dest=u'LAX', distance=2475, flight=21, hour=19, min=2, month=1, origin=u'JFK', tailnum=u'N327AA', year=2014),
 Row(air_time=350, arr_delay=1, arr_time=1706, cancelled=u'0', carrier=u'AA', day=1, dep_delay=2, dep_time=1347, dest=u'LAX', distance=2475, flight=117, hour=13, min=47, month=1, origin=u'JFK', tailnum=u'N319AA', year=2014),
 Row(air_time=339, arr_delay=0, arr_time=2145, cancelled=u'0', carrier=u'AA', day=1, dep_delay=4, dep_ti

In [8]:
flightsDF.where("origin = 'JFK' AND dest = 'MIA'").count()

2750

In [9]:
flightsDF.where("origin = 'JFK' AND dest = 'MIA'").limit(2).collect()

[Row(air_time=161, arr_delay=-17, arr_time=1828, cancelled=u'0', carrier=u'AA', day=1, dep_delay=-1, dep_time=1509, dest=u'MIA', distance=1089, flight=145, hour=15, min=9, month=1, origin=u'JFK', tailnum=u'N5FJAA', year=2014),
 Row(air_time=166, arr_delay=-8, arr_time=1227, cancelled=u'0', carrier=u'AA', day=1, dep_delay=7, dep_time=917, dest=u'MIA', distance=1089, flight=1085, hour=9, min=17, month=1, origin=u'JFK', tailnum=u'N5DWAA', year=2014)]

### Count the flights that departed early and arrived late

In [10]:
flightsDF.where("dep_delay < 0 AND arr_delay > 0").count()

30239

### Find the flightwith the longest arrival delay

In [11]:
sqlContext.sql("select * from flight order by arr_delay desc").take(1)

[Row(air_time=200, arr_delay=1494, arr_time=1008, cancelled=u'0', carrier=u'AA', day=4, dep_delay=1498, dep_time=727, dest=u'DFW', distance=1372, flight=1381, hour=7, min=27, month=10, origin=u'EWR', tailnum=u'N4WJAA', year=2014)]

In [12]:
flightsDF.sort(flightsDF.arr_delay.desc()).take(1)

[Row(air_time=200, arr_delay=1494, arr_time=1008, cancelled=u'0', carrier=u'AA', day=4, dep_delay=1498, dep_time=727, dest=u'DFW', distance=1372, flight=1381, hour=7, min=27, month=10, origin=u'EWR', tailnum=u'N4WJAA', year=2014)]

### Find the top 10 destinations ordered by the number of flights

In [13]:
sqlContext.sql("select dest, count(1) as number from flight group by dest order by number desc").take(10)

[Row(dest=u'LAX', number=14434),
 Row(dest=u'ATL', number=12808),
 Row(dest=u'SFO', number=11907),
 Row(dest=u'MCO', number=11709),
 Row(dest=u'BOS', number=11609),
 Row(dest=u'ORD', number=11589),
 Row(dest=u'MIA', number=9928),
 Row(dest=u'CLT', number=9624),
 Row(dest=u'FLL', number=9471),
 Row(dest=u'DCA', number=6748)]

In [14]:
from pyspark.sql.functions import *
flightsDF.groupBy('dest').count().orderBy(desc('count')).take(10)

[Row(dest=u'LAX', count=14434),
 Row(dest=u'ATL', count=12808),
 Row(dest=u'SFO', count=11907),
 Row(dest=u'MCO', count=11709),
 Row(dest=u'BOS', count=11609),
 Row(dest=u'ORD', count=11589),
 Row(dest=u'MIA', count=9928),
 Row(dest=u'CLT', count=9624),
 Row(dest=u'FLL', count=9471),
 Row(dest=u'DCA', count=6748)]

### Find top 10 destinations with the worst avg arrival delay, ignoring flights that arrived early

In [15]:
sqlContext.sql("select dest, avg(arr_delay) as avg_delay from flight where arr_delay > 0 group by dest order by avg_delay desc").take(10)

[Row(dest=u'EGE', avg_delay=80.59016393442623),
 Row(dest=u'AVP', avg_delay=67.0),
 Row(dest=u'JAC', avg_delay=59.666666666666664),
 Row(dest=u'CAK', avg_delay=57.907042253521126),
 Row(dest=u'TUL', avg_delay=56.99152542372882),
 Row(dest=u'MSN', avg_delay=54.80237154150198),
 Row(dest=u'IAD', avg_delay=53.324599708879184),
 Row(dest=u'BGR', avg_delay=52.73504273504273),
 Row(dest=u'OKC', avg_delay=52.14503816793893),
 Row(dest=u'TVC', avg_delay=51.15151515151515)]

In [16]:
flightsDF.where('arr_delay > 0').groupBy('dest').avg('arr_delay').orderBy(desc('avg(arr_delay)')).take(10)

[Row(dest=u'EGE', avg(arr_delay)=80.59016393442623),
 Row(dest=u'AVP', avg(arr_delay)=67.0),
 Row(dest=u'JAC', avg(arr_delay)=59.666666666666664),
 Row(dest=u'CAK', avg(arr_delay)=57.907042253521126),
 Row(dest=u'TUL', avg(arr_delay)=56.99152542372882),
 Row(dest=u'MSN', avg(arr_delay)=54.80237154150198),
 Row(dest=u'IAD', avg(arr_delay)=53.324599708879184),
 Row(dest=u'BGR', avg(arr_delay)=52.73504273504273),
 Row(dest=u'OKC', avg(arr_delay)=52.14503816793893),
 Row(dest=u'TVC', avg(arr_delay)=51.15151515151515)]

### Take a sample of 1% of the flights and then calculate the average departure delay for that sample

In [17]:
flightsDF.sample(False,0.01).registerTempTable("sample_flight")
sqlContext.sql("select avg(dep_delay) from sample_flight").collect()

[Row(avg(dep_delay)=12.206664022213408)]

In [18]:
flightsDF.sample(False,0.01).groupBy().avg('dep_delay').collect()

[Row(avg(dep_delay)=14.033333333333333)]

### For all flights from JFK during June, show the average departure delay for each destination

In [19]:
sqlContext.sql("select dest, avg(dep_delay) as delay from flight where origin='JFK' and month=6 group by dest order by delay desc").collect()

[Row(dest=u'MSP', delay=28.69047619047619),
 Row(dest=u'SMF', delay=22.96551724137931),
 Row(dest=u'OAK', delay=22.75862068965517),
 Row(dest=u'DEN', delay=19.083333333333332),
 Row(dest=u'JAX', delay=18.49425287356322),
 Row(dest=u'CVG', delay=17.571428571428573),
 Row(dest=u'SAT', delay=17.17241379310345),
 Row(dest=u'PSE', delay=17.133333333333333),
 Row(dest=u'BUR', delay=16.428571428571427),
 Row(dest=u'SLC', delay=14.965909090909092),
 Row(dest=u'RSW', delay=14.913793103448276),
 Row(dest=u'BNA', delay=14.172413793103448),
 Row(dest=u'ATL', delay=13.491620111731844),
 Row(dest=u'IAD', delay=13.333333333333334),
 Row(dest=u'BOS', delay=13.006864988558352),
 Row(dest=u'BUF', delay=12.900826446280991),
 Row(dest=u'SAN', delay=12.286713286713287),
 Row(dest=u'SJC', delay=12.241379310344827),
 Row(dest=u'ORD', delay=12.136054421768707),
 Row(dest=u'CLT', delay=12.104803493449781),
 Row(dest=u'CMH', delay=11.862068965517242),
 Row(dest=u'SYR', delay=11.656862745098039),
 Row(dest=u'SEA

In [20]:
flightsDF.where("origin = 'JFK' and month = 6").groupBy('dest').avg('dep_delay').orderBy(desc('avg(dep_delay)')).collect()

[Row(dest=u'MSP', avg(dep_delay)=28.69047619047619),
 Row(dest=u'SMF', avg(dep_delay)=22.96551724137931),
 Row(dest=u'OAK', avg(dep_delay)=22.75862068965517),
 Row(dest=u'DEN', avg(dep_delay)=19.083333333333332),
 Row(dest=u'JAX', avg(dep_delay)=18.49425287356322),
 Row(dest=u'CVG', avg(dep_delay)=17.571428571428573),
 Row(dest=u'SAT', avg(dep_delay)=17.17241379310345),
 Row(dest=u'PSE', avg(dep_delay)=17.133333333333333),
 Row(dest=u'BUR', avg(dep_delay)=16.428571428571427),
 Row(dest=u'SLC', avg(dep_delay)=14.965909090909092),
 Row(dest=u'RSW', avg(dep_delay)=14.913793103448276),
 Row(dest=u'BNA', avg(dep_delay)=14.172413793103448),
 Row(dest=u'ATL', avg(dep_delay)=13.491620111731844),
 Row(dest=u'IAD', avg(dep_delay)=13.333333333333334),
 Row(dest=u'BOS', avg(dep_delay)=13.006864988558352),
 Row(dest=u'BUF', avg(dep_delay)=12.900826446280991),
 Row(dest=u'SAN', avg(dep_delay)=12.286713286713287),
 Row(dest=u'SJC', avg(dep_delay)=12.241379310344827),
 Row(dest=u'ORD', avg(dep_delay)=

### For every origin/dest pair, count the number of flights

In [21]:
sqlContext.sql("select origin, dest, count(1) as number from flight group by origin, dest order by origin, number desc").collect()

[Row(origin=u'EWR', dest=u'SFO', number=4539),
 Row(origin=u'EWR', dest=u'BOS', number=4268),
 Row(origin=u'EWR', dest=u'LAX', number=4226),
 Row(origin=u'EWR', dest=u'ATL', number=4182),
 Row(origin=u'EWR', dest=u'MCO', number=4164),
 Row(origin=u'EWR', dest=u'CLT', number=3921),
 Row(origin=u'EWR', dest=u'ORD', number=3272),
 Row(origin=u'EWR', dest=u'IAH', number=3073),
 Row(origin=u'EWR', dest=u'FLL', number=2909),
 Row(origin=u'EWR', dest=u'DEN', number=2243),
 Row(origin=u'EWR', dest=u'DFW', number=2224),
 Row(origin=u'EWR', dest=u'DTW', number=2149),
 Row(origin=u'EWR', dest=u'MIA', number=2094),
 Row(origin=u'EWR', dest=u'PHX', number=2002),
 Row(origin=u'EWR', dest=u'PBI', number=1874),
 Row(origin=u'EWR', dest=u'TPA', number=1833),
 Row(origin=u'EWR', dest=u'BNA', number=1797),
 Row(origin=u'EWR', dest=u'STL', number=1740),
 Row(origin=u'EWR', dest=u'LAS', number=1555),
 Row(origin=u'EWR', dest=u'MDW', number=1537),
 Row(origin=u'EWR', dest=u'MSP', number=1485),
 Row(origin=u

In [22]:
flightsDF.groupBy(['origin','dest']).count().orderBy(['origin','count'],ascending=[1,0]).collect()

[Row(origin=u'EWR', dest=u'SFO', count=4539),
 Row(origin=u'EWR', dest=u'BOS', count=4268),
 Row(origin=u'EWR', dest=u'LAX', count=4226),
 Row(origin=u'EWR', dest=u'ATL', count=4182),
 Row(origin=u'EWR', dest=u'MCO', count=4164),
 Row(origin=u'EWR', dest=u'CLT', count=3921),
 Row(origin=u'EWR', dest=u'ORD', count=3272),
 Row(origin=u'EWR', dest=u'IAH', count=3073),
 Row(origin=u'EWR', dest=u'FLL', count=2909),
 Row(origin=u'EWR', dest=u'DEN', count=2243),
 Row(origin=u'EWR', dest=u'DFW', count=2224),
 Row(origin=u'EWR', dest=u'DTW', count=2149),
 Row(origin=u'EWR', dest=u'MIA', count=2094),
 Row(origin=u'EWR', dest=u'PHX', count=2002),
 Row(origin=u'EWR', dest=u'PBI', count=1874),
 Row(origin=u'EWR', dest=u'TPA', count=1833),
 Row(origin=u'EWR', dest=u'BNA', count=1797),
 Row(origin=u'EWR', dest=u'STL', count=1740),
 Row(origin=u'EWR', dest=u'LAS', count=1555),
 Row(origin=u'EWR', dest=u'MDW', count=1537),
 Row(origin=u'EWR', dest=u'MSP', count=1485),
 Row(origin=u'EWR', dest=u'SEA', c